In [4]:
import torch
from transformers import LayoutLMForTokenClassification, TrainingArguments, Trainer


c:\Users\21269\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from datasets import load_from_disk

train_dataset = load_from_disk('dataset/preprocessed_data/train')
test_dataset = load_from_disk('dataset/preprocessed_data/test')


In [1]:
import torch

print("CUDA Available: ", torch.cuda.is_available())
print("Number of GPUs: ", torch.cuda.device_count())
if torch.cuda.is_available():
    print("CUDA Device Name: ", torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA Available:  True
Number of GPUs:  1
CUDA Device Name:  NVIDIA GeForce GTX 1650 with Max-Q Design


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
labels = ["O", "B-HEADER", "I-HEADER", "B-QUESTION", "I-QUESTION", "B-ANSWER", "I-ANSWER"]


model = LayoutLMForTokenClassification.from_pretrained("microsoft/layoutlm-base-uncased", num_labels=len(labels))
model.to(device)

Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LayoutLMForTokenClassification(
  (layoutlm): LayoutLMModel(
    (embeddings): LayoutLMEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (x_position_embeddings): Embedding(1024, 768)
      (y_position_embeddings): Embedding(1024, 768)
      (h_position_embeddings): Embedding(1024, 768)
      (w_position_embeddings): Embedding(1024, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LayoutLMEncoder(
      (layer): ModuleList(
        (0-11): 12 x LayoutLMLayer(
          (attention): LayoutLMAttention(
            (self): LayoutLMSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
 

In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

c:\Users\21269\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


In [11]:
trainer.train()


  9%|▉         | 10/114 [00:32<06:56,  4.01s/it]

{'loss': 1.3372, 'grad_norm': 7.078808784484863, 'learning_rate': 1.824561403508772e-05, 'epoch': 0.26}


 18%|█▊        | 20/114 [01:10<05:43,  3.66s/it]

{'loss': 0.2312, 'grad_norm': 0.8434969782829285, 'learning_rate': 1.649122807017544e-05, 'epoch': 0.53}


 26%|██▋       | 30/114 [01:46<04:55,  3.51s/it]

...

 96%|█████████▋| 110/114 [07:35<00:18,  4.67s/it]

{'loss': 0.0014, 'grad_norm': 0.028959989547729492, 'learning_rate': 7.017543859649123e-07, 'epoch': 2.89}


                                                 
100%|██████████| 114/114 [08:03<00:00,  4.24s/it]

{'eval_loss': 0.0009335149079561234, 'eval_runtime': 8.9011, 'eval_samples_per_second': 5.617, 'eval_steps_per_second': 1.46, 'epoch': 3.0}
{'train_runtime': 483.8902, 'train_samples_per_second': 0.924, 'train_steps_per_second': 0.236, 'train_loss': 0.1405073140927574, 'epoch': 3.0}


TrainOutput(global_step=114, training_loss=0.1405073140927574, metrics={'train_runtime': 483.8902, 'train_samples_per_second': 0.924, 'train_steps_per_second': 0.236, 'total_flos': 117615921638400.0, 'train_loss': 0.1405073140927574, 'epoch': 3.0})

In [12]:
metrics = trainer.evaluate()
print(metrics)


100%|██████████| 13/13 [00:08<00:00,  1.52it/s]

{'eval_loss': 0.0009335149079561234, 'eval_runtime': 8.5938, 'eval_samples_per_second': 5.818, 'eval_steps_per_second': 1.513, 'epoch': 3.0}


In [13]:
model.save_pretrained("./fine-tuned-layoutlm")
